<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/12_RESOLVE_ecoregions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract RESOLVE Ecoregions

## Import gee and authenticate

In [ ]:
import ee
import geemap

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

layer_name = "RS_039"

ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(aoi)
output_name = f"{layer_name}_{aoi_name}"


## Process and extract Ecoregion data

In [ ]:
# Data set contains linestrings and polygons. Export can only handle one geometry

# Filter out line strings and only keep multipolygons (change the ee.Filter.equals to "Polygon" if there are no multiparts)

# Map over the features and set the 'geometry_type' property (then filter to only include desired geoms)
polygons = ecoregions.map(
    lambda f: ee.Feature(f).set('geometry_type', ee.Feature(f).geometry().type())
).filter(ee.Filter.equals('geometry_type', 'MultiPolygon'))

ecoregions = ee.FeatureCollection(polygons)


In [ ]:
task = ee.batch.Export.table.toDrive(
    collection = ecoregions,
    description = "EXPORT IMAGE TO DRIVE",
    fileFormat = "SHP",
    fileNamePrefix = output_name,
    folder = "GEE_exports"
    )

task.start()

In [ ]:
task.status()

## Ad hoc analysis

Used this for developing the PACE buffer

In [ ]:
mkr = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
aoi = ee.FeatureCollection("WWF/HydroATLAS/v1/Basins/level06").filterBounds(mkr)
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(aoi)

In [ ]:
# Filter out line strings and only keep multipolygons (change the ee.Filter.equals to "Polygon" if there are no multiparts)

# Map over the features and set the 'geometry_type' property (then filter to only include desired geoms)
polygons = ecoregions.map(
    lambda f: ee.Feature(f).set('geometry_type', ee.Feature(f).geometry().type())
).filter(ee.Filter.equals('geometry_type', 'MultiPolygon'))

eco_polygons = ee.FeatureCollection(polygons)

In [ ]:
Map = geemap.Map()
Map.addLayer(aoi, {}, "aoi")
Map.addLayer(eco_polygons, {}, "ecoregions")
Map.center_object(eco_polygons, zoom = 10)
Map

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection = eco_polygons,
    description = "EXPORT IMAGE TO DRIVE",
    fileFormat = "SHP",
    fileNamePrefix = "ecoregions_basin_level06",
    folder = "GEE_exports"
    )

task.start()

In [ ]:
task.status()